In [124]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from twilio.rest import Client

STOCK = "TSLA"
COMPANY_NAME = "Tesla Inc"

AV_API_KEY = "YOUR OWN API KEY"
## STEP 1: Use https://www.alphavantage.co
# When STOCK price increase/decreases by 5% between yesterday and the day before yesterday then print("Get News").

alphavantage_url = 'https://www.alphavantage.co/query'
alphavantage_parameter = {
    "function": "TIME_SERIES_DAILY",
    "symbol": STOCK,
    "apikey": AV_API_KEY,
}

In [117]:
r = requests.get(alphavantage_url, params=alphavantage_parameter)
data = r.json()

In [118]:
sample_data = data["Time Series (Daily)"]

sample_data = pd.DataFrame.from_dict(sample_data, orient="index")
df = sample_data.head(2)
print(df)

             1. open   2. high    3. low  4. close  5. volume
2024-02-16  202.0600  203.1700  197.4000  199.9500  111346705
2024-02-15  189.1600  200.8800  188.8595  200.4500  120831762


In [130]:
ALERT_PERCENTAGE_CHANGE = 0.1

last_close = float(df.iloc[0]["4. close"])
previous_close = float(df.iloc[1]["4. close"])

rate_change = round((last_close - previous_close) / previous_close * 100, 3)

if abs(rate_change) >= ALERT_PERCENTAGE_CHANGE:
    print(f"Stock price changed {rate_change}%.\nAlarm trigger % change is {ALERT_PERCENTAGE_CHANGE}%\nGET NEWS.")
    get_news = True

Stock price changed -0.249%.
Alarm trigger % change is 0.1%
GET NEWS.


In [127]:
## STEP 2: Use https://newsapi.org
# Instead of printing ("Get News"), actually get the first 3 news pieces for the COMPANY_NAME. 

# News API
NEWS_API_KEY = "YOUR OWN API"

# TWILIO
account_sid = "YOUR OWN API"
auth_token = "YOUR OWN TOKEN"
client = Client(account_sid, auth_token)

if get_news: 
       date = date.today() - timedelta(3)
       print(date)
       news_url = ('https://newsapi.org/v2/everything?'
              f'q={COMPANY_NAME}&'
              f'from={date}&'
              'sortBy=relevancy&'
              f'apiKey={NEWS_API_KEY}')
       response = requests.get(news_url)
       


2024-02-16


In [136]:
top_news = response.json()["articles"][0:3]
## STEP 3: Use https://www.twilio.com
# Send a seperate message with the percentage change and each article's title and description to your phone number. 
if rate_change >= 0:
    up_down = "🔺"
else: up_down = "🔻"
    
for news in top_news:
    msg = f"{STOCK}: {up_down}{rate_change}\nHeadlne: {news['title']}\nBrief: {top_news[0]['description']}\nLink: {news['url']}"
    print(msg)
    message = client.messages \
            .create(
                 body=msg,
                 from_='Phone Number',
                 to='+Phone Number'
             )
    

TSLA: 🔻-0.249
Headlne: Jeff Bezos sells more Amazon stock, bringing total to $6 billion this month
Brief: Billionaire Jeff Bezos has sold another $2 billion in Amazon.com Inc. shares, bringing his total in stock sales over the past week or so to about $6 billion.
Link: https://www.marketwatch.com/story/jeff-bezos-sells-more-amazon-stock-bringing-total-to-6-billion-this-month-d2942c60
TSLA: 🔻-0.249
Headlne: Based on hundreds of years of bubbles, this is what may finally halt AI, Magnificent Seven stock run
Brief: Billionaire Jeff Bezos has sold another $2 billion in Amazon.com Inc. shares, bringing his total in stock sales over the past week or so to about $6 billion.
Link: https://www.marketwatch.com/story/based-on-hundreds-of-years-of-bubbles-this-is-what-may-finally-halt-ai-magnificent-seven-stock-run-c37ea7c3
TSLA: 🔻-0.249
Headlne: S&P 500 seen grinding to new high with producer prices, sentiment data ahead
Brief: Billionaire Jeff Bezos has sold another $2 billion in Amazon.com Inc.

In [133]:
#Optional: Format the SMS message like this: 
"""
TSLA: 🔺2%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
or
"TSLA: 🔻5%
Headline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. 
Brief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds' and investors' portfolio positions as of March 31st, near the height of the coronavirus market crash.
"""



'\nTSLA: 🔺2%\nHeadline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. \nBrief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds\' and investors\' portfolio positions as of March 31st, near the height of the coronavirus market crash.\nor\n"TSLA: 🔻5%\nHeadline: Were Hedge Funds Right About Piling Into Tesla Inc. (TSLA)?. \nBrief: We at Insider Monkey have gone over 821 13F filings that hedge funds and prominent investors are required to file by the SEC The 13F filings show the funds\' and investors\' portfolio positions as of March 31st, near the height of the coronavirus market crash.\n'